In [ ]:
import pickle
import numpy as np
import matplotlib.tri as tri
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
suffix = '20250129'

In [ ]:
parentrepo = "/home/shreejit.jadhav/WORK/RC/ligo-commissioning-modeling"
repopath = f"{parentrepo}/analysis/O4/Indep_Param_Study"

maxtems = 8
datapath = f'{repopath}/data/run_{suffix}'
figpath = f'{repopath}/plots/plots_{suffix}'

data = {}
for f in glob(f'{datapath}/*.pkl'):
    with open(f, "rb") as file:
        dat1 = pickle.load(file)
    
    for k in dat1:
        if k not in data:
            data[k] = np.array(dat1[k])
        data[k] = np.append(data[k], dat1[k])

In [ ]:
ITMXlens_f = data["ITMXlens.f"]
ITMYlens_f = data["ITMYlens.f"]
ITMX_Rcx = data["ITMX.Rcx"]
PRG = data["PRG"]

unique_rcx = np.unique(ITMX_Rcx)

for rcx in unique_rcx:

    mask = ITMX_Rcx == rcx
    x = ITMXlens_f[mask]
    y = ITMYlens_f[mask]
    z = PRG[mask]

    # triangulation object
    try:
        triang = tri.Triangulation(x, y)
    except:
        continue

    # contour plotting
    xi = np.linspace(np.min(x), np.max(x), 100)
    yi = np.linspace(np.min(y), np.max(y), 100)
    xi, yi = np.meshgrid(xi, yi)

    # interpolate z values
    interpolator = tri.LinearTriInterpolator(triang, z)
    zi = interpolator(xi, yi)

    # plot
    plt.figure()
    contour = plt.contourf(xi, yi, zi, levels=20, cmap='viridis')
    plt.colorbar(contour, label='PRG')

    plt.title(f'Contour Plot of ITMXlens.f vs ITMYlens.f (ITMX.Rcx = {rcx})')
    plt.xlabel('ITMXlens.f')
    plt.ylabel('ITMYlens.f')
    plt.grid()
    plt.savefig(f'{repopath}/plots/plots_{suffix}/contour_plot_rcx_{rcx:.2f}.png')
    plt.show()